# Rate of Tweets & Volatility

Does the daily volume of tweets about a stock have any correlation with its intraday volatility?

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import t as t_dist

pd.set_option('display.float_format', '{:.10f}'.format)

In [2]:
tweets_df = pd.read_csv('../tweets-data/stock_tweets.csv')

In [3]:
tickers = ['TSLA', 'MSFT', 'PG', 'META', 'AMZN', 'GOOG', 'AMD', 'AAPL',
            'NFLX', 'TSM', 'KO', 'F', 'COST', 'DIS', 'VZ', 'CRM', 'INTC', 'BA',
            'BX', 'NOC', 'PYPL', 'ENPH', 'NIO', 'ZS', 'XPEV']

Use Pearson Correlation Coefficient formula to determine correlation between the tweet rate and stock volatility.

$
r = \frac{\sum_{i=1}^{n} (x_i - \bar{x}) (y_i - \bar{y})}{\sqrt{\sum_{i=1}^{n} (x_i - \bar{x})^2} \cdot \sqrt{\sum_{i=1}^{n} (y_i - \bar{y})^2}}
$

The Pearson correlation coefficient measures the strength and direction of a linear relationship between two numerical variables.

So, a greater magnitude of $r$ represents a higher degree of correlation.

In [4]:
def pearson_correlation(x, y):
    x = np.array(x)
    y = np.array(y)
    
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    
    top = np.sum((x- x_mean) * (y - y_mean))
    bottom = np.sqrt(np.sum((x - x_mean)**2)) * np.sqrt(np.sum((y - y_mean)**2))
    
    
    return top/bottom

Get p-value from the correlation coefficient.

$
t = r \cdot \sqrt{\frac{n-2}{1- r^2}}
$

$
p = 2 \cdot (1 - t.cdf(|t |, df = n-2))
$

The p value is the probability of observing a correlation as extreme as r, by chance, if the true correlation were zero.

In [5]:
def pearson_p_value(r, n):
    if n < 3 or np.isnan(r):
        return np.nan
    t_stat = r * np.sqrt((n-2) / (1 - r**2))
    p = 2 * (1 - t_dist.cdf(np.abs(t_stat), df = n-2))
    return p

In [11]:
def analyze_ticker(ticker, tweets_df=tweets_df, stock_folder='stock-data'):
    # Load price data, and then get measurement of volatility
    price_path = f"../{stock_folder}/{ticker}.csv"
    price_df = pd.read_csv(price_path, skiprows=[1, 2])
    price_df.columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
    price_df['Date'] = pd.to_datetime(price_df['Date'])
    price_df['Volatility'] = (price_df['High'] - price_df['Low']) / price_df['Open']
    
    # Load tweet data
    tweet_df = tweets_df[tweets_df['Stock Name'] == ticker].copy()
    tweet_df['Date'] = pd.to_datetime(tweet_df['Date']).dt.date
    tweet_counts = tweet_df.groupby('Date').size().reset_index(name='TweetCount') # Create df of number of tweets per day
    tweet_counts['Date'] = pd.to_datetime(tweet_counts['Date'])
    tweet_counts = tweet_counts[tweet_counts['Date'].isin(price_df['Date'])] # Get rid of tweet count info on non-trading days
    
    # Merge
    merged = pd.merge(tweet_counts, price_df[['Date', 'Volatility']], on='Date', how='inner')
    
    # Same-day correlation
    r = pearson_correlation(merged['TweetCount'], merged['Volatility'])
    p = pearson_p_value(r, len(merged))
    
    # Lagged-correlation: Analyzing volatility the week after tweets
    merged['Volatility_Tomorrow'] = merged['Volatility'].shift(-7)
    r_lag = pearson_correlation(merged['TweetCount'][:-7], merged['Volatility'][:-7])
    p_lag = pearson_p_value(r_lag, len(merged) - 7)
    
    return {
        'Ticker': ticker,
        'N': len(merged),
        'SameDayCorr': r,
        'SameDayP': p,
        'LaggedCorr': r_lag,
        'LaggedP': p_lag
    }

Run for all tickers

In [9]:
results = [analyze_ticker(ticker, tweets_df) for ticker in tickers]
results_df = pd.DataFrame(results)

In [10]:
results_df

,Ticker,N,SameDayCorr,SameDayP,LaggedCorr,LaggedP
0,TSLA,252,0.3634974002,0.0000000027,0.3616285795,0.0000000055
1,MSFT,252,0.1715172149,0.0063436128,0.1751706188,0.0059755490
2,PG,252,0.0935873416,0.1384680023,0.0992399502,0.1213255398
3,META,219,0.1892728931,0.0049479721,0.1877218836,0.0061154294
4,AMZN,252,0.2052306052,0.0010503312,0.2094102919,0.0009749212
5,GOOG,238,0.2050253761,0.0014718561,0.2077016590,0.0015016936
6,AMD,249,0.1936873300,0.0021400100,0.2012728753,0.0016488220
7,AAPL,252,0.2255161524,0.0003076665,0.2054670217,0.0012198544
8,NFLX,228,0.3770625876,0.0000000041,0.3856397549,0.0000000030
9,TSM,252,0.1692781361,0.0070746094,0.1635079487,0.0103625423


In [12]:
results_df.to_csv('correlation_results.csv', index=False)